# Microsoft NNI Rrtiarii Example

This example will show Retiarii's ability to **express** and **explore** the model space for Neural Architecture Search and Hyper-Parameter Tuning in a simple way.

Let's start the journey with Retiarii!

## Step 1: Express the Model Space

Model space is defined by users to express a set of models that they want to explore, which contains potentially good-performing models. In Retiarii framework, a model space is defined with two parts: a base model and possible mutations on the base model.

### Step 1.1: Define the Base Model

Defining a base model is almost the same as defining a PyTorch (or TensorFlow) model. Usually, you only need to replace the code ``import torch.nn as nn`` with ``import nni.retiarii.nn.pytorch as nn`` to use NNI wrapped PyTorch modules. Below is a very simple example of defining a base model.

In [ ]:
import torch.nn.functional as F
import nni.retiarii.nn.pytorch as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3, padding=1)
        self.conv3 = nn.Conv2d(16, 16, 1)

        self.bn = nn.BatchNorm2d(16)

        self.gap = nn.AdaptiveAvgPool2d(4)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        bs = x.size(0)

        x = self.pool(F.relu(self.conv1(x)))
        x0 = F.relu(self.conv2(x))
        x1 = F.relu(self.conv3(x0))

        x1 += x0
        x = self.pool(self.bn(x1))

        x = self.gap(x).view(bs, -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = Net()

### Step 1.2: Define the Model Mutations

A base model is only one concrete model, not a model space. NNI provides APIs and primitives for users to express how the base model can be mutated, i.e., a model space that includes many models. The following will use inline Mutation APIs ``LayerChoice`` to choose a layer from candidate operations and use ``InputChoice`` to try out skip connection.

In [1]:
import torch.nn.functional as F
import nni.retiarii.nn.pytorch as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # self.conv1 = nn.Conv2d(3, 6, 3, padding=1)
        self.conv1 = nn.LayerChoice([nn.Conv2d(3, 6, 3, padding=1), nn.Conv2d(3, 6, 5, padding=2)])
        self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 3, padding=1)
        self.conv2 = nn.LayerChoice([nn.Conv2d(6, 16, 3, padding=1), nn.Conv2d(6, 16, 5, padding=2)])
        self.conv3 = nn.Conv2d(16, 16, 1)

        self.skipconnect = nn.InputChoice(n_candidates=2)
        self.bn = nn.BatchNorm2d(16)

        self.gap = nn.AdaptiveAvgPool2d(4)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        bs = x.size(0)

        x = self.pool(F.relu(self.conv1(x)))
        x0 = F.relu(self.conv2(x))
        x1 = F.relu(self.conv3(x0))

        x1 = self.skipconnect([x1, x1+x0])
        x = self.pool(self.bn(x1))

        x = self.gap(x).view(bs, -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = Net()

Besides inline mutations, Retiarii also provides ``mutator``, a more general approach to express complex model space.

## Step 2: Explore the Model Space

With a defined model space, users can explore the space in two ways. One is the classic NAS method, which searchs by evaluating each sampled model independently. The other is using one-shot weight-sharing based search, which consumes much less computational resource compared to the first one. 

## Part 1: Classic NAS method

We will demo ths classic NAS method first. In the classic NAS process, the search strategy repeatedly generates new models, and the model evaluator is for training and validating each generated model. The obtained performance of a generated model is collected and sent to the search strategy for generating better models. 

Users can choose a proper search strategy to explore the model space, and use a chosen or user-defined model evaluator to evaluate the performance of each sampled model.

### Step 2.1: Choose or Write a Search Strategy

Currently, Retiarii supports many common strategies, such as Random, Regularized evolution and TPE, etc. According to the APIs of Retiarii, you can also customize a new strategy easily, and there we use the TPE strategy as an example.

In [2]:
import nni.retiarii.strategy as strategy

simple_strategy = strategy.TPEStrategy() # choice: Random, GridSearch, RegularizedEvolution, TPEStrategy

### Step 2.2: Choose or Write a Model Evaluator

The model evaluator should correctly identify the use case of the model and the optimization goal. For example, on a classification task, an <input, label> dataset is needed, the loss function could be cross entropy and the optimized metric could be the accuracy.

Retiarii provides two ways for users to write a new model evaluator. In the context of PyTorch, Retiarii has provided two built-in model evaluators, designed for simple use cases: classification and regression. These two evaluators are built upon the awesome library PyTorch-Lightning. Here we take a classification task on CIFAR10 dataset as an example.

In [3]:
from torchvision import transforms
from torchvision.datasets import CIFAR10
from nni.retiarii import serialize
import nni.retiarii.evaluator.pytorch.lightning as pl

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = serialize(CIFAR10, root="./data", train=True, download=True, transform=transform)
test_dataset = serialize(CIFAR10, root="./data", train=False, download=True, transform=transform)

trainer = pl.Classification(train_dataloader=pl.DataLoader(train_dataset, batch_size=64),
                            val_dataloaders=pl.DataLoader(test_dataset, batch_size=64),
                            max_epochs=2, gpus=[0])

Files already downloaded and verified
Files already downloaded and verified


GPU available: True, used: True


[2021-05-15 14:24:13] INFO (lightning/MainThread) GPU available: True, used: True


TPU available: None, using: 0 TPU cores


[2021-05-15 14:24:13] INFO (lightning/MainThread) TPU available: None, using: 0 TPU cores


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


[2021-05-15 14:24:13] INFO (lightning/MainThread) LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


As the model evaluator is running in another process (possibly in some remote machines), the defined evaluator, along with all its parameters, needs to be correctly serialized. For example, users should use the dataloader that has been already wrapped as a serializable class defined in ``nni.retiarii.evaluator.pytorch.lightning``. For the arguments used in dataloader, recursive serialization needs to be done, until the arguments are simple types like int, str, float.

### Step 2.3: Configure the Experiment

After all the above are prepared, it is time to configure an experiment to do the model search. The basic experiment configuration is as follows.

NNI allows users to run experiments in different training platforms to speed up the search, like  Local Machine, Remote Servers, OpenPAI, Kubeflow, FrameworkController on K8S, DLWorkspace, Azure Machine Learning, AdaptDL, other cloud options, and even Hybrid mode. There we use the local mode with GPU speeding up.

In [4]:
from nni.retiarii.experiment.pytorch import RetiariiExeConfig, RetiariiExperiment

exp = RetiariiExperiment(model, trainer, [], simple_strategy)

exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = 'Retiarii example'
exp_config.trial_concurrency = 2
exp_config.max_trial_number = 6
exp_config.trial_gpu_number = 2
exp_config.max_experiment_duration = '5m'
exp_config.training_service.use_active_gpu = True

### Step 2.4: Run and View the Experiment

You can launch the experiment now! 

In [5]:
exp.run(exp_config, 8745)

[2021-05-15 14:25:20] INFO (nni.experiment/MainThread) Creating experiment, Experiment ID: r8nsjvky
[2021-05-15 14:25:20] INFO (nni.experiment/MainThread) Connecting IPC pipe...
[2021-05-15 14:25:21] INFO (nni.experiment/MainThread) Starting web server...
[2021-05-15 14:25:22] INFO (nni.experiment/MainThread) Setting up...
[2021-05-15 14:25:22] INFO (nni.runtime.msg_dispatcher_base/Thread-6) Dispatcher started
[2021-05-15 14:25:22] INFO (nni.retiarii.experiment.pytorch/MainThread) Web UI URLs: http://127.0.0.1:8745 http://10.190.173.211:8745 http://172.17.0.1:8745
[2021-05-15 14:25:22] INFO (nni.retiarii.experiment.pytorch/MainThread) Start strategy...
[2021-05-15 14:25:22] INFO (nni.retiarii.strategy.tpe_strategy/MainThread) TPE strategy has been started.
[2021-05-15 14:25:22] INFO (hyperopt.tpe/MainThread) tpe_transform took 0.002444 seconds
[2021-05-15 14:25:22] INFO (hyperopt.tpe/MainThread) TPE using 0 trials
[2021-05-15 14:25:22] INFO (hyperopt.tpe/MainThread) tpe_transform took 

/home/v-yiruxu/anaconda3/envs/envs/lib/python3.8/site-packages/json_tricks/nonp.py:221: JsonTricksDeprecation: `json_tricks.load(s)` stripped some comments, but `ignore_comments` was not passed; in the next major release, the behaviour when `ignore_comments` is not passed will change; it is recommended to explicitly pass `ignore_comments=True` if you want to strip comments; see https://github.com/mverleg/pyjson_tricks/issues/74
  warnings.warn('`json_tricks.load(s)` stripped some comments, but `ignore_comments` was '
GPU available: True, used: True


[2021-05-15 14:26:17] INFO (lightning/Thread-5) GPU available: True, used: True


TPU available: None, using: 0 TPU cores


[2021-05-15 14:26:17] INFO (lightning/Thread-5) TPU available: None, using: 0 TPU cores


...

[2021-05-15 14:28:09] INFO (lightning/Thread-5) GPU available: True, used: True


TPU available: None, using: 0 TPU cores


[2021-05-15 14:28:09] INFO (lightning/Thread-5) TPU available: None, using: 0 TPU cores


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


[2021-05-15 14:28:09] INFO (lightning/Thread-5) LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
Files already downloaded and verified
Files already downloaded and verified
[2021-05-15 14:28:12] INFO (nni.retiarii.experiment.pytorch/Thread-7) Stopping experiment, please wait...
[2021-05-15 14:28:12] INFO (nni.runtime.msg_dispatcher_base/Thread-6) Dispatcher exiting...
[2021-05-15 14:28:12] INFO (nni.retiarii.experiment.pytorch/MainThread) Strategy exit
[2021-05-15 14:28:12] INFO (nni.retiarii.experiment.pytorch/MainThread) Waiting for experiment to become DONE (you can ctrl+c if there is no running trial jobs)...
[2021-05-15 14:28:13] INFO (nni.retiarii.experiment.pytorch/Thread-7) Experiment stopped
[2021-05-15 14:28:15] INFO (nni.runtime.msg_dispatcher_base/Thread-6) Dispatcher terminiated


Besides, NNI provides WebUI to help users view the experiment results and make more advanced analysis.

### Step 2.5: Export the top Model

After searching, exporting the top model script is also very convenient.

In [6]:
print('Final model:')
for model_code in exp.export_top_models():
    print(model_code)

Final model:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import nni.retiarii.nn.pytorch

print('hhhh')
print(torch.cuda.device_count())


import torch
import nni


class _model__conv1(nn.Module):
    def __init__(self):
        super().__init__()
        self.layerchoice_layerchoice_1_0 = torch.nn.modules.conv.Conv2d(padding=1, in_channels=3, out_channels=6, kernel_size=3)

    def forward(self, *_inputs):
        layerchoice_layerchoice_1_0 = self.layerchoice_layerchoice_1_0(_inputs[0])
        return layerchoice_layerchoice_1_0



class _model__conv2(nn.Module):
    def __init__(self):
        super().__init__()
        self.layerchoice_layerchoice_2_1 = torch.nn.modules.conv.Conv2d(padding=2, in_channels=6, out_channels=16, kernel_size=5)

    def forward(self, *_inputs):
        layerchoice_layerchoice_2_1 = self.layerchoice_layerchoice_2_1(_inputs[0])
        return layerchoice_layerchoice_2_1



class _model(nn.Module):
    def 

## Part 2: One-shot NAS method

In this part, we focus on this one-shot approach. The principle of the One-shot approach is combining all the models in a model space into one big model (usually called super-model or super-graph). It takes charge of both search, training and testing, by training and evaluating this big model.

Retiarii has supported some classic one-shot trainers, like DARTS trainer, ENAS trainer, ProxylessNAS trainer, Single-path trainer, and users can customize a new one-shot trainer according to the APIs provided by Retiarii conveniently.

Here, we show an example to use DARTS trainer manually.

In [ ]:
import torch
from utils import accuracy

from nni.retiarii.oneshot.pytorch import DartsTrainer

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

trainer = DartsTrainer(
    model=model,
    loss=criterion,
    metrics=lambda output, target: accuracy(output, target),
    optimizer=optimizer,
    num_epochs=2,
    dataset=train_dataset,
    batch_size=64,
    log_frequency=10
    )

trainer.fit()

Similarly, the optimal structure found can be exported.

In [ ]:
print('Final architecture:', trainer.export())